In [2]:
import pandas as pd
import astropy.units as u
from astropy.coordinates import Distance
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

import bilby
import bilby.gw.utils as ut

In [3]:
# import the samples
samples = pd.read_csv('params_for_SNR.csv', index_col=0)
# convert our redshifts into luminosity distances
# samples['dl'] = (samples['redshift']* astropy.cosmology.units.redshift).to(u.Mpc, astropy.cosmoloy.units.redshift_distance(None, kind="luminosity")) 
d = Distance(unit=u.Mpc, z = samples['redshift'], cosmology = None).value
dl = np.asarray(d)
samples = samples.assign(dl = dl)
samples = samples.assign(mass_1_det = samples['mass_1']*(1+samples['redshift']))
samples = samples.assign(mass_2_det = samples['mass_2']*(1+samples['redshift']))
samples

,mass_1,mass_ratio,a_1,a_2,cos_tilt_1,cos_tilt_2,redshift,mass_2,chi_eff,tilt_1,...,sin_tilt_1,sin_tilt_2,chi_p,spin1z,spin2z,spin1x,spin2x,dl,mass_1_det,mass_2_det
0,14.206189,0.753297,0.084464,0.150304,-0.261760,0.392646,0.906271,10.701485,0.012746,1.835642,...,0.965133,0.919690,0.100027,-0.022109,0.059016,0.081519,0.138233,6016.435895,27.080843,20.399927
1,7.631592,0.987168,0.301969,0.514466,-0.610410,-0.613718,1.837756,7.533664,-0.249606,2.227374,...,0.792086,0.789526,0.400233,-0.184325,-0.315737,0.239185,0.406184,14360.479388,21.656593,21.378695
2,6.088737,0.993740,0.235612,0.437827,0.873063,0.821194,2.090188,6.050621,0.282381,0.509347,...,0.487607,0.570649,0.248059,0.205704,0.359541,0.114886,0.249846,16804.436694,18.815344,18.697560
3,13.368498,0.656860,0.519649,0.126563,0.212090,-0.385168,1.802572,8.781230,0.047193,1.357083,...,0.977250,0.922846,0.507827,0.110213,-0.048748,0.507827,0.116798,14024.688026,37.466181,24.610031
4,23.997822,0.479771,0.313633,0.007355,0.270814,0.404514,1.759925,11.513460,0.058363,1.296558,...,0.962632,0.914532,0.301913,0.084936,0.002975,0.301913,0.006726,13619.365717,66.232183,31.776282
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
994,9.782304,0.941944,0.027176,0.547558,-0.020054,0.942494,0.650648,9.214381,0.250040,1.590851,...,0.999799,0.334224,0.170916,-0.000545,0.516070,0.027171,0.183007,4011.292700,16.147137,15.209696
995,10.076905,0.942602,0.514802,0.481686,0.691731,0.687116,1.820907,9.498514,0.343911,0.806913,...,0.722155,0.726548,0.371767,0.356105,0.330974,0.371767,0.349968,14199.515059,28.426008,26.794420
996,6.066189,0.994744,0.373819,0.606443,0.462265,0.051141,0.806717,6.034305,0.102095,1.090249,...,0.886742,0.998691,0.602013,0.172803,0.031014,0.331481,0.605650,5215.339092,10.959885,10.902279
997,5.792281,0.996428,0.227619,0.346497,-0.669760,-0.677548,0.823380,5.771592,-0.193536,2.304681,...,0.742578,0.735478,0.253801,-0.152450,-0.234769,0.169025,0.254841,5347.752174,10.561531,10.523807


**To do:**
* check how much difference an increased sampling frequency makes on the ultimate SNR
    * max freq will be half sampling_freq
    * plus safety factor
* duration
    * duration was 1 second
        * this produced an error for being a lower length of time in the detector than the length of the waveform
        * for a waveform of 42.7 seconds, a duration of 50 and 100 both produce the same SNR (3sf)
            * a little off from lal
            * this should be more accurate
            * peters and mathews inspiral papers 1963, 64
        * find a way to automate the choice of this
            * calculate_time_to_merger
    * inspiral length +2s
    * nominally for merger and ringdown
    * work in powers of 2 (bayeswave psds)
    * need a simple way of determining the chirp length in order to control this
        * this is done with the calculate_time_to_merger

* what are:
    * phi_12: Azimuthal angle between the two component spins
    * phi_jl: Azimuthal angle between the total binary angular momentum and the orbital angular momentum
        * wobbles with precession so changes
        * phi in orbital plane
        * uniform on 0,2pi or -pi,pi
    * theta_jn: Angle between the total binary angular momentum and the line of sight
        * this looks like the inclination, no this is theta_ln, iota
        * j is total ang mom
        * jn is angle between j and n(los)
        * uniform in cos, isotropic rather than uniform , on 0, pi
    * psi, polarisation
        * uniform
    * phi, phase
        * uniform
        * degeneracy between these
* check and plot the psd
* also look into the reference frequency
    * "Reference frequency at which the spins are calculated"
* parallelising on condor
    * notes should already be on the git site
* check if I have symmetric mass ratio or mass ratio
* check if my masses are source or detector frame
    * it seems they are source frame
    * https://colmtalbot.github.io/gwpopulation/GWTC1.html?highlight=detector%20frame in this population model the masses are source frame
    * I've sampled from the population models so my masses will be in the frame of those models, which should be source frame
    * looking for concrete confirmation of this

In [4]:
# specify other terms
np.random.seed(220222)
N = len(samples['mass_1'])
theta_jn = np.arccos(np.random.uniform(-1,1,N))
psi = np.random.uniform(0,np.pi,N)
phase = np.random.uniform(0,2*np.pi,N)
ra = np.random.uniform(0,2*np.pi,N)
dec = np.random.uniform(-0.5*np.pi,0.5*np.pi,N)
phi_12 = np.random.uniform(0,2*np.pi,N)
phi_jl = np.random.uniform(0,2*np.pi,N)

In [5]:
duration = np.zeros(N)
for i in range(0,N,1):
    duration[i]=np.int(ut.calculate_time_to_merger(20,samples['mass_1'][i],samples['mass_2'][i], samples['chi_eff'][i]))

/data/wiay/johnny/phd1/phd1/lib/python3.7/site-packages/ipykernel_launcher.py:3: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  This is separate from the ipykernel package so we can avoid doing imports until


In [6]:
duration[349]

1.0

In [30]:
# code from bilby tutorial
import bilby
import bilby.gw.utils as ut
import logging
import time
logger = bilby.core.utils.logger
logger.setLevel('WARNING')

H1_snr = np.zeros(N-1)
L1_snr = np.zeros(N-1)

for i in range(0,N-1,1):
    #  Define injection parameters

    injection_dict=dict(mass_1=samples['mass_1_det'][i], mass_ratio=samples['mass_ratio'][i], redshift=samples['redshift'][i], 
        theta_jn=theta_jn[i], psi=psi[1], phase=phase[i], geocent_time=1242442967.46, 
        ra=ra[i], dec=dec[i], a_1=samples['a_1'][i], a_2=samples['a_2'][i], 
        phi_12=phi_12[i], phi_jl=phi_jl[i], 
        tilt_1=samples['tilt_1'][i], tilt_2=samples['tilt_2'][i])


    #  Define time and frequency parameters

    sampling_frequency=4096
    duration=np.int(ut.calculate_time_to_merger(minimum_frequency,samples['mass_1'][i],samples['mass_2'][i], samples['chi_eff'][i]))
    trigger_time=injection_dict['geocent_time'] #- duration
    minimum_frequency=10

    # Sometimes the duration above 20Hz (or chosen frequency) is zero. In these cases I set the SNR to none.
    if duration is 0:
        H1_snr[i] = None

    else:

        #  Setup waveform generator

        waveform_arguments = dict(waveform_approximant='IMRPhenomPv2',
                                reference_frequency=20., minimum_frequency=minimum_frequency)
        waveform_generator = bilby.gw.WaveformGenerator(
                duration=duration, sampling_frequency=sampling_frequency,
                frequency_domain_source_model=bilby.gw.source.lal_binary_black_hole,
                waveform_arguments=waveform_arguments,
                parameter_conversion=bilby.gw.conversion.convert_to_lal_binary_black_hole_parameters)


        #  Setup interferometers and inject signal

        ifos = bilby.gw.detector.InterferometerList(['H1', 'L1'])
        for ifo in ifos:
            ifo.minimum_frequency = minimum_frequency
            ifo.maximum_frequency = sampling_frequency/2.
            # in controlling the psd, a separate file would need specified for LIGO (aLIGO) and Virgo (AdV)
            #ifo.power_spectral_density = bilby.gw.detector.PowerSpectralDensity(psd_file='AdV_psd.txt')

        ifos.set_strain_data_from_power_spectral_densities(
            sampling_frequency=sampling_frequency, duration=duration,
            start_time=(trigger_time - duration))
        # try:
        #     ifos.inject_signal(waveform_generator=waveform_generator, parameters=injection_dict)
        # except:
        #     print(injection_dict)
        #     print(duration)
        #     continue
        ifos.inject_signal(waveform_generator=waveform_generator, parameters=injection_dict)

        # Access injection data
        H1_snr[i] = ifos.meta_data['H1']['optimal_SNR']
        L1_snr[i] = ifos.meta_data['L1']['optimal_SNR']

        #time.sleep(0.1)


/data/wiay/johnny/phd1/phd1/lib/python3.7/site-packages/ipykernel_launcher.py:25: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations


In [35]:
np.where(H1_snr == 0)

(array([998]),)

### Issues
* Sometimes the duration above 20Hz is zero. In these cases I set the SNR to none.
* Had some NaNs in the mass_2. This was due to m2 being to close to the allnowed range set by mmin in draw_from_posterior

### Next
* this is slow as a loop so set up the data file with multiple injections and see if that's faster.

In [9]:
print(H1_snr)

[3.18677308 1.00081026 1.039424   0.78446669 1.35140034 0.46775213
 0.87735913 1.22047508 2.1872062  0.18165289 0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0. 

In [10]:
len(H1_snr)

999

In [11]:
ut.calculate_time_to_merger(minimum_frequency,samples['mass_1'][62],samples['mass_2'][62], samples['chi_eff'][62])

1.120430469887806

In [12]:
samples['mass_1'][62]

36.52524080573456

In [13]:
samples['mass_2'][62]

19.581766745830304